# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import re
import nltk
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from sqlalchemy import create_engine

from gensim.test.utils import datapath
from gensim import utils
import gensim.models

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import joblib

[nltk_data] Downloading package punkt to /home/tarciso/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tarciso/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tarciso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [189]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
messages_df = pd.read_sql_table(con=engine, table_name='Message')
categories_df = pd.read_sql_table(con=engine, table_name='MessageCategoryWide')
X = messages_df.message.values
Y_df = categories_df.drop('message_id', axis=1)
Y = Y_df.values
category_columns = Y_df.columns

In [135]:
categories = pd.read_csv('../data/disaster_categories.csv')
categories['categories_list'] = categories['categories'].apply(lambda x: re.sub(r"[_a-z]+-0[;]?", "", x)) \
                                    .apply(lambda x: re.sub(r"-1", "", x)) \
                                    .apply(lambda x: re.sub(r";$", "", x))

In [137]:
categories

,id,categories,categories_list
0,2,related-1;request-0;offer-0;aid_related-0;medi...,related
1,7,related-1;request-0;offer-0;aid_related-1;medi...,related;aid_related;other_aid;weather_related;...
2,8,related-1;request-0;offer-0;aid_related-0;medi...,related
3,9,related-1;request-1;offer-0;aid_related-1;medi...,related;request;aid_related;medical_products;o...
4,12,related-1;request-0;offer-0;aid_related-0;medi...,related
...,...,...,...
26243,30261,related-0;request-0;offer-0;aid_related-0;medi...,
26244,30262,related-0;request-0;offer-0;aid_related-0;medi...,
26245,30263,related-1;request-0;offer-0;aid_related-0;medi...,related
26246,30264,related-1;request-0;offer-0;aid_related-1;medi...,related;aid_related;military


In [118]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [4]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [5]:
category_columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 4. Split dataset
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=199)

In [7]:
X_train

array(['crudess you are the most charm and charismatic DT member .. thanks for all your videos in Santiago.. and to talk with me twice',
       'Tidal waves triggered by an earthquake off Indonesia on Sunday swept over a vast swath of coastlines, including those in India, Indonesia, Malaysia, the Maldives, Somalia, Sri Lanka and Thailand.',
       'The earthquake happened at 4h34 in the evening and for the night. ',
       ...,
       'Earthquake of the day #Haiti or Google possibly leaving China ?',
       'When the matter had been debated previously, the Non-Aligned Movement foreign ministers firmly stated that there was no right to humanitarian intervention.',
       '@DumboNYC : Oh no RT @endtwist : ConEd power station in #DUMBO is flooding ! #nopower #sandy'],
      dtype=object)

In [8]:
y_train

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [9]:
print(X_train.shape, y_train.shape)

(19608,) (19608, 36)


### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]

    return clean_tokens

In [11]:
def tokenize_str(text):
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]
    #Return tokens list as a string joined by whitespaces
    clean_tokens_str = ' '.join(clean_tokens)

    return clean_tokens_str

In [12]:
tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [13]:
tokenize_str(X[0])

'weather update cold front cuba could pas haiti'

### 3. Generate tokenized messages table for further analysis

In [14]:
messages_df['tokens'] = messages_df.message.apply(lambda x: tokenize_str(x))

In [15]:
messages_df

,message_id,message,original,genre,num_words,tokens
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,13,weather update cold front cuba could pas haiti
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,9,hurricane
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,6,looking someone name
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,13,un report leogane 80 90 destroyed hospital st ...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,12,say west side haiti rest country today tonight
...,...,...,...,...,...,...
26139,30261,The training demonstrated how to enhance micro...,None,news,21,training demonstrated enhance micronutrient pr...
26140,30262,A suitable candidate has been selected and OCH...,None,news,22,suitable candidate selected ocha jakarta curre...
26141,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,23,proshika operating cox bazar municipality 5 un...
26142,30264,"Some 2,000 women protesting against the conduc...",None,news,31,2 000 woman protesting conduct election tearga...


In [16]:
messages_tokens = messages_df[['message_id','tokens']]

In [17]:
messages_tokens.to_sql('MessageTokens', engine, index=False, if_exists='replace')

In [18]:
messages_categories = messages_tokens.merge(categories_df, on='message_id')

In [19]:
messages_categories.head()

,message_id,tokens,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,weather update cold front cuba could pas haiti,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,hurricane,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,looking someone name,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,un report leogane 80 90 destroyed hospital st ...,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,say west side haiti rest country today tonight,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3. Find n-grams and save to DB

In [20]:
def get_ngrams_freqs(messages_array, n=1):
    vec = CountVectorizer(ngram_range=(n, n)).fit(messages_array)
    bag_of_words = vec.transform(messages_array)
    word_count = bag_of_words.sum(axis=0)
    words_freq = [(word, n, word_count[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[2], reverse=True)
    words_freq_df = pd.DataFrame(data = words_freq, columns = ['ngram','n','count'])
    return words_freq_df

In [21]:
unigrams_freqs = get_ngrams_freqs(messages_tokens.tokens)
bigrams_freqs = get_ngrams_freqs(messages_tokens.tokens, n=2)
trigrams_freqs = get_ngrams_freqs(messages_tokens.tokens, n=3)
ngrams_freqs = pd.concat([unigrams_freqs, bigrams_freqs, trigrams_freqs])

In [22]:
ngrams_freqs

,ngram,n,count
0,water,1,3034
1,people,1,2998
2,food,1,2892
3,help,1,2649
4,need,1,2484
...,...,...,...
289130,crisis identifying crisis,3,1
289131,identifying crisis function,3,1
289132,crisis function hiv,3,1
289133,function hiv aid,3,1


In [23]:
ngrams_freqs.to_sql('NGramsFreqs', engine, index=False, if_exists='replace')

### 4. Train a Word2Vec model using messages text

In [24]:
class Message(object):
    """An interator that yields messages tokens (lists of str)."""
    
    def __init__(self, messages_df, sample_size=-1):
        self.messages_df = messages_df
        self.sample_size = sample_size

    def __iter__(self):
        messages = messages_df
        
        #Sample dataset if specified
        if self.sample_size > 0:
            messages = messages.sample(self.sample_size)
        
        #Yeld tokenized message joined by whitespaces
        for token_str in messages.tokens:
            yield token_str.split()

In [25]:
messages_df.tokens

0           weather update cold front cuba could pas haiti
1                                                hurricane
2                                     looking someone name
3        un report leogane 80 90 destroyed hospital st ...
4           say west side haiti rest country today tonight
                               ...                        
26139    training demonstrated enhance micronutrient pr...
26140    suitable candidate selected ocha jakarta curre...
26141    proshika operating cox bazar municipality 5 un...
26142    2 000 woman protesting conduct election tearga...
26143    radical shift thinking came result meeting rec...
Name: tokens, Length: 26144, dtype: object

In [26]:
sentences = Message(messages_df=messages_df)
w2v_model_full = gensim.models.Word2Vec(sentences=sentences)
w2v_model_full.save("messages-word2vec-full.model")

In [27]:
w2v_model_full = gensim.models.Word2Vec.load("messages-word2vec-full.model")

In [28]:
#?w2v_model_full

In [29]:
for i, word in enumerate(w2v_model_full.wv.vocab):
    if i == 10:
        break
    print(word)

weather
update
cold
front
cuba
could
pas
haiti
hurricane
looking


In [30]:
w2v_model_full.wv['earthquake']

array([-0.4580385 , -0.27376625, -0.73521465,  0.37736434,  0.93309957,
        0.39935714, -0.49234065,  0.9419288 , -0.25134334, -1.3529804 ,
        0.14977379,  0.46156198,  0.84126824, -0.04854123, -0.4594544 ,
       -0.26190546, -0.37590995,  0.40064278,  0.01681725,  0.27057725,
        0.39359283,  0.20490897, -0.20667851, -0.0345754 ,  0.6368317 ,
        0.05800449, -0.6153193 , -1.0990179 ,  0.2303953 ,  0.9355079 ,
        1.019853  ,  0.5140892 ,  0.07158959,  0.40510416, -0.03284523,
        0.1792508 , -0.1518288 , -0.15107486, -0.23644361, -0.9310771 ,
        0.03854028, -0.4627645 ,  0.60985947, -0.28503847,  0.45460945,
        0.4666094 ,  1.3113381 ,  0.5432689 ,  0.39310548, -0.7705728 ,
       -0.07419207, -0.9764262 ,  1.0725194 , -0.37089157, -0.6078079 ,
       -0.5047722 ,  0.03854486, -0.00536926,  0.5899817 ,  0.29847994,
        0.67140335, -0.63053477, -0.66853267, -1.0385207 , -0.416342  ,
        0.6471132 , -0.33853984,  0.24877574, -0.151765  , -0.73

##### Implement Vector Aggregator

In [31]:
np.array([1,2])[0]

1

In [32]:
len(X)

26144

In [33]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [34]:
X.shape[0]

26144

In [35]:
from collections import defaultdict

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model
        self.num_dims = word2vec_model.vector_size
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings
    
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model
        self.num_dims = word2vec_model.vector_size
        self.word_weights = None
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        
        tfidf_weights = [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        self.word_weights = defaultdict(lambda: max_idf, tfidf_weights)
    
        return self
    
    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w]*self.word_weights[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [36]:
test_dict = defaultdict(lambda: 1, [("Hello" , 7), ("hi" , 10), ("there" , 45),("at" , 23),("this" , 77)])
test_dict['Hi']

1

##### Use a Pipeline to train the Word2Vec estimator

In [37]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

w2v_mean_pip = Pipeline([
    ('w2v_mean', TfidfEmbeddingVectorizer(w2v_model_full)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

In [38]:
%%time

w2v_mean_pip.fit(X_train, y_train)

CPU times: user 12.4 s, sys: 13.6 ms, total: 12.5 s
Wall time: 12.5 s


Pipeline(memory=None,
         steps=[('w2v_mean',
                 <__main__.TfidfEmbeddingVectorizer object at 0x7fa51f489890>),
                ('clf',
                 MultiOutputClassifier(estimator=GaussianNB(priors=None,
                                                            var_smoothing=1e-09),
                                       n_jobs=None))],
         verbose=False)

In [39]:
%%time

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
y_pred = w2v_mean_pip.predict(X_test)

#print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=category_columns))
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

              precision    recall  f1-score   support

   related-0       0.26      0.72      0.38      1532
   related-1       0.81      0.31      0.45      4950

   micro avg       0.42      0.41      0.42      6482
   macro avg       0.53      0.51      0.41      6482
weighted avg       0.68      0.41      0.43      6482

              precision    recall  f1-score   support

   request-0       0.83      0.94      0.88      5393
   request-1       0.22      0.08      0.12      1143

    accuracy                           0.79      6536
   macro avg       0.52      0.51      0.50      6536
weighted avg       0.72      0.79      0.75      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.14      0.25      6506
     offer-1       0.01      1.00      0.01        30

    accuracy                           0.14      6536
   macro avg       0.50      0.57      0.13      6536
weighted avg       1.00      0.14      0.25      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

 other_aid-0       0.86      0.97      0.91      5660
 other_aid-1       0.08      0.02      0.03       876

    accuracy                           0.84      6536
   macro avg       0.47      0.49      0.47      6536
weighted avg       0.76      0.84      0.79      6536

                          precision    recall  f1-score   support

infrastructure_related-0       0.95      0.12      0.22      6111
infrastructure_related-1       0.07      0.90      0.12       425

                accuracy                           0.17      6536
               macro avg       0.51      0.51      0.17      6536
            weighted avg       0.89      0.17      0.21      6536

              precision    recall  f1-score   support

 transport-0       0.97      0.10      0.19      6219
 transport-1       0.05      0.93      0.10       317

    accuracy                           0.14      6536
   macro avg       0.51      0.52      0.14      6536
we

### 5. Create Estimator to both train and average w2v on corpus data

In [40]:
from gensim.sklearn_api import W2VTransformer
from gensim.test.utils import common_texts

model = W2VTransformer(size=10, min_count=1, seed=1)

model.fit(common_texts)
          
model.transform(['graph', 'system'])

array([[ 0.0245881 ,  0.03953072, -0.01233948, -0.00461599,  0.00405706,
        -0.01307748,  0.01215516, -0.01282117,  0.03348592, -0.02948483],
       [ 0.03473149,  0.01462194,  0.0262618 , -0.02464902, -0.01954006,
        -0.00992026, -0.01438491, -0.03246041,  0.0375623 ,  0.04060492]],
      dtype=float32)

In [41]:
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [42]:
messages_df.head(5).tokens.str.split().values

array([list(['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']),
       list(['hurricane']), list(['looking', 'someone', 'name']),
       list(['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']),
       list(['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'])],
      dtype=object)

In [43]:
messages_df.head(5).tokens

0       weather update cold front cuba could pas haiti
1                                            hurricane
2                                 looking someone name
3    un report leogane 80 90 destroyed hospital st ...
4       say west side haiti rest country today tonight
Name: tokens, dtype: object

In [44]:
w2v_model = W2VTransformer()
w2v_model.fit(messages_df.tokens.str.split().values)

W2VTransformer(alpha=0.025, batch_words=10000, cbow_mean=1,
               hashfxn=<built-in function hash>, hs=0, iter=5,
               max_vocab_size=None, min_alpha=0.0001, min_count=5, negative=5,
               null_word=0, sample=0.001, seed=1, sg=0, size=100,
               sorted_vocab=1, trim_rule=None, window=5, workers=3)

In [46]:
word = messages_df.tokens[0].split()[1]
print(word)
w2v_model.transform(word)

update


array([[-0.1420873 , -0.33508325, -0.16770744, -0.1500786 ,  0.16726807,
         0.15470034, -0.15004374,  0.02010204, -0.15072422, -0.24947192,
         0.08761428,  0.19374529,  0.23780076, -0.19996251, -0.12466462,
        -0.35647163, -0.23258583,  0.12307049,  0.03470751,  0.21371457,
         0.15902479,  0.04242009, -0.13418187, -0.2920171 , -0.2743935 ,
        -0.1678258 , -0.31443968, -0.4825736 , -0.04308612,  0.10380849,
         0.2893048 ,  0.19040187, -0.44116718,  0.14310774, -0.39174768,
        -0.08584639, -0.11778536,  0.08137684, -0.11216237, -0.12758714,
         0.1291606 , -0.15629181,  0.26040673,  0.15458855,  0.13721748,
         0.41650438,  0.41433838,  0.128733  ,  0.05030085, -0.3072067 ,
        -0.11070127, -0.17349747,  0.1707144 , -0.01139092,  0.03244712,
        -0.111556  ,  0.1723215 , -0.16479649,  0.12299413,  0.0843934 ,
        -0.00946739, -0.02679778, -0.3235797 , -0.25301677, -0.1923044 ,
         0.02924292,  0.0450475 , -0.01411721, -0.0

In [47]:
test_model = gensim.models.Word2Vec(messages_df.tokens.str.split().values)

In [48]:
for i, word in enumerate(test_model.wv.vocab):
    if i == 10:
        break
    print(word)

weather
update
cold
front
cuba
could
pas
haiti
hurricane
looking


In [49]:
test_model.wv['earthquake']

array([-0.5085411 , -0.42081872, -0.9078778 ,  0.42672846,  1.1348137 ,
        0.5843697 , -0.6246567 ,  1.0815555 , -0.34175354, -1.576989  ,
        0.21889104,  0.48355803,  0.80581886, -0.12138204, -0.53289   ,
       -0.31697255, -0.3198318 ,  0.33583292,  0.05101398,  0.3707698 ,
        0.4516033 ,  0.2722424 , -0.1295693 ,  0.01182473,  0.7488737 ,
        0.22579421, -0.6172903 , -1.172379  ,  0.31454954,  1.0397036 ,
        1.0652086 ,  0.54922324, -0.10604451,  0.48277193, -0.1388124 ,
        0.11439073, -0.17948535, -0.15480489, -0.25017682, -0.9688826 ,
       -0.00636338, -0.49018678,  0.56302357, -0.43850434,  0.47368705,
        0.4227974 ,  1.3574692 ,  0.47287682,  0.29438028, -0.8669572 ,
       -0.07454896, -0.98171985,  1.1369545 , -0.51176876, -0.7067084 ,
       -0.58569956, -0.00428575,  0.12038623,  0.65997046,  0.3211721 ,
        0.79170585, -0.7234073 , -0.6351506 , -1.0500762 , -0.48675495,
        0.6651924 , -0.45518294,  0.26012543, -0.15297022, -0.77

In [327]:
class MeanEmbeddingTrainVectorizer(object):

    def __init__(self, word2vec_model=None):
        if word2vec_model is None:
            self.word2vec_model = None
            self.num_dims = -1
        else:
            self.word2vec_model = word2vec_model
            self.num_dims = word2vec_model.vector_size
            
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.word2vec_model = gensim.models.Word2Vec(X)
            self.num_dims = self.word2vec_model.vector_size
            
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [328]:
class TfidfEmbeddingTrainVectorizer(object):
    
    def __init__(self, word2vec_model=None):
        if word2vec_model is None:
            self.word2vec_model = None
            self.num_dims = -1
        else:
            self.word2vec_model = word2vec_model
            self.num_dims = word2vec_model.vector_size
        self.word_weights = None
        
        
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.word2vec_model = gensim.models.Word2Vec(X)
            self.num_dims = self.word2vec_model.vector_size
            
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        
        tfidf_weights = [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        self.word_weights = defaultdict(lambda: max_idf, tfidf_weights)
    
        return self
    
    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w]*self.word_weights[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [52]:
%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

              precision    recall  f1-score   support

   related-0       0.39      0.04      0.07      1532
   related-1       0.76      0.93      0.84      4950

   micro avg       0.76      0.72      0.74      6482
   macro avg       0.58      0.48      0.45      6482
weighted avg       0.68      0.72      0.66      6482

              precision    recall  f1-score   support

   request-0       0.83      0.96      0.89      5393
   request-1       0.28      0.07      0.12      1143

    accuracy                           0.80      6536
   macro avg       0.55      0.52      0.50      6536
weighted avg       0.73      0.80      0.76      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.99      0.99      6506
     offer-1       0.02      0.03      0.03        30

    accuracy                           0.99      6536
   macro avg       0.51      0.51      0.51      6536
weighted avg       0.99      0.99      0.99      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

 transport-0       0.97      0.12      0.22      6219
 transport-1       0.05      0.94      0.10       317

    accuracy                           0.16      6536
   macro avg       0.51      0.53      0.16      6536
weighted avg       0.93      0.16      0.21      6536

              precision    recall  f1-score   support

 buildings-0       0.97      0.12      0.22      6209
 buildings-1       0.05      0.92      0.10       327

    accuracy                           0.16      6536
   macro avg       0.51      0.52      0.16      6536
weighted avg       0.92      0.16      0.21      6536

               precision    recall  f1-score   support

electricity-0       0.98      0.11      0.19      6414
electricity-1       0.02      0.89      0.04       122

     accuracy                           0.12      6536
    macro avg       0.50      0.50      0.11      6536
 weighted avg       0.96      0.12      0.19      6536

             

### 6. Use Glove pre-trained model to generate feature vectors

In [53]:
# from gensim.test.utils import datapath, get_tmpfile
# from gensim.models import KeyedVectors
# from gensim.scripts.glove2word2vec import glove2word2vec

# glove_file = 'glove-pretrained/glove.6B.50d.txt'
# tmp_file = 'glove-pretrained/glove.6B.50d_word2vec.txt'

# _ = glove2word2vec(glove_file, tmp_file)

# model = KeyedVectors.load_word2vec_format(tmp_file)

In [54]:
from gensim.models import KeyedVectors

glove_w2v = KeyedVectors.load_word2vec_format('glove-pretrained/glove.6B.50d_word2vec.txt', binary=False)

##### Use a Pipeline to train the Word2Vec estimator

In [55]:
%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('w2v_tfidf', TfidfEmbeddingTrainVectorizer(glove_w2v)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


              precision    recall  f1-score   support

   related-0       0.33      0.02      0.03      1532
   related-1       0.76      0.96      0.85      4950

   micro avg       0.76      0.73      0.74      6482
   macro avg       0.55      0.49      0.44      6482
weighted avg       0.66      0.73      0.65      6482

              precision    recall  f1-score   support

   request-0       0.83      0.95      0.89      5393
   request-1       0.23      0.07      0.10      1143

    accuracy                           0.80      6536
   macro avg       0.53      0.51      0.50      6536
weighted avg       0.72      0.80      0.75      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.98      0.99      6506
     offer-1       0.00      0.00      0.00        30

    accuracy                           0.97      6536
   macro avg       0.50      0.49      0.49      6536
weighted avg       0.99      0.97      0.98      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          precision    recall  f1-score   support

infrastructure_related-0       0.96      0.13      0.23      6111
infrastructure_related-1       0.07      0.92      0.13       425

                accuracy                           0.18      6536
               macro avg       0.51      0.52      0.18      6536
            weighted avg       0.90      0.18      0.22      6536

              precision    recall  f1-score   support

 transport-0       0.97      0.14      0.25      6219
 transport-1       0.05      0.93      0.10       317

    accuracy                           0.18      6536
   macro avg       0.51      0.53      0.17      6536
weighted avg       0.93      0.18      0.24      6536

              precision    recall  f1-score   support

 buildings-0       0.97      0.14      0.24      6209
 buildings-1       0.05      0.92      0.10       327

    accuracy                           0.18      6536
   macro avg       0.51      0.53      0.17      6536
we

### 7. Use Doc2Vec to generate feature vectors

In [56]:
messages_df['message'].apply(lambda x: len(x.split(' '))).sum()

631288

In [57]:
messages_categories

,message_id,tokens,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,weather update cold front cuba could pas haiti,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,hurricane,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,looking someone name,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,un report leogane 80 90 destroyed hospital st ...,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,say west side haiti rest country today tonight,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26139,30261,training demonstrated enhance micronutrient pr...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26140,30262,suitable candidate selected ocha jakarta curre...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26141,30263,proshika operating cox bazar municipality 5 un...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26142,30264,2 000 woman protesting conduct election tearga...,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
d2v_train, d2v_test = train_test_split(messages_categories, test_size=0.3, random_state=42)

In [144]:
d2v_test.head()

,message_id,tokens,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
22578,26234,unhcr also distributing tent plastic sheet sle...,1,0,0,1,0,1,0,0,...,0,0,1,1,0,0,0,0,0,0
24096,27942,mission found idp already started return home ...,1,0,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
19278,22569,meulaboh currently providing water sanitation ...,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
17825,20955,fuel efficient stove also lower health risk re...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13712,16390,effort required affected country monitor curre...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
print(d2v_train.shape)
print(d2v_test.shape)

(18300, 38)
(7844, 38)


In [60]:
d2v_train.head(1).drop(['message_id','tokens'], axis=1).sum(axis=1)

9757    0
dtype: int64

In [61]:
def read_messages(df, tokens_only=False):
    for index, row in df.iterrows():
        tokens = row.tokens.split()
        tags = row[2:].values
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(tokens, tags)

train_corpus = list(read_messages(d2v_train))
test_corpus = list(read_messages(d2v_test, tokens_only=True))

In [62]:
train_corpus

[TaggedDocument(words=['note', 'message', 'important', 'message', 'miss', 'enough', 'information'], tags=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object)),
 TaggedDocument(words=['friend', 'life', 'carrefour', 'haiti', 'say', 'hasnt', 'aid', 'passing', 'people', 'area', 'cant', 'find', 'anything', 'eat'], tags=array([1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], dtype=object)),
 TaggedDocument(words=['street', 'three', '3', 'child', 'lost', 'posse', '75', 'delma'], tags=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object)),
 TaggedDocument(words=['month', 'northeast', 'sri', 'lanka', 'writhed', 'grip', 'clash', 'rebel', 'government', 'force', 'soldier', 'occupying', 'house', 'hiding', 'behind', 'sandbag', 'bunker', 'sign', 'life', 'deserted', 't

In [63]:
test_corpus

[['unhcr',
  'also',
  'distributing',
  'tent',
  'plastic',
  'sheet',
  'sleeping',
  'mat',
  'hygiene',
  'sanitation',
  'item',
  'unflooded',
  'area',
  'leitchuor',
  'nip',
  'nip'],
 ['mission',
  'found',
  'idp',
  'already',
  'started',
  'return',
  'home',
  'prepare',
  'land',
  'next',
  'season',
  'plant',
  'dry',
  'season',
  'crop',
  'wetland',
  'area'],
 ['meulaboh',
  'currently',
  'providing',
  'water',
  'sanitation',
  'service',
  '15',
  '000',
  'people',
  'existing',
  'settlement',
  'camp',
  'developed',
  'relocate',
  'people',
  'public',
  'place',
  'school',
  'semi',
  'permanent',
  'relocation',
  'camp'],
 ['fuel',
  'efficient',
  'stove',
  'also',
  'lower',
  'health',
  'risk',
  'reducing',
  'unhealthy',
  'smoke',
  'particle',
  'emission',
  'reduces',
  'cooking',
  'time',
  'heat',
  'concentrated'],
 ['effort',
  'required',
  'affected',
  'country',
  'monitor',
  'current',
  'infestation',
  'undertake',
  'necessa

In [64]:
d2v_model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=5, epochs=10)

In [65]:
d2v_model.build_vocab(train_corpus)

In [66]:
%%time 

d2v_model.train(train_corpus, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

CPU times: user 56.5 s, sys: 2.84 s, total: 59.4 s
Wall time: 39.4 s


In [86]:
vector = d2v_model.docvecs.similarity('only you can prevent forest fires','only you can prevent forest fires')
vector

KeyError: "tag 'only you can prevent forest fires' not seen in training corpus/invalid"

#### Create a Doc2Vec Estimator/Transformer 

In [77]:
tokenize(str(X_train))

['crude',
 'charm',
 'charismatic',
 'dt',
 'member',
 'thanks',
 'video',
 'santiago',
 'talk',
 'twice',
 'tidal',
 'wave',
 'triggered',
 'earthquake',
 'indonesia',
 'sunday',
 'swept',
 'vast',
 'swath',
 'coastline',
 'including',
 'india',
 'indonesia',
 'malaysia',
 'maldives',
 'somalia',
 'sri',
 'lanka',
 'thailand',
 'earthquake',
 'happened',
 '4h34',
 'evening',
 'night',
 'earthquake',
 'day',
 'haiti',
 'google',
 'possibly',
 'leaving',
 'china',
 'matter',
 'debated',
 'previously',
 'non',
 'aligned',
 'movement',
 'foreign',
 'minister',
 'firmly',
 'stated',
 'right',
 'humanitarian',
 'intervention',
 'dumbonyc',
 'oh',
 'rt',
 'endtwist',
 'coned',
 'power',
 'station',
 'dumbo',
 'flooding',
 'nopower',
 'sandy']

In [78]:
y_train

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [185]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.base import BaseEstimator
import multiprocessing

class Doc2VecTransformer(BaseEstimator):

    def __init__(self, vector_size=100, epochs=20):
        self.epochs = epochs
        self.vector_size = vector_size
        self.workers = multiprocessing.cpu_count() - 1
        self.d2v_model = None

    def fit(self, X, y):
        tagged_x = [TaggedDocument(tokens_str.split(), [index]) for index, tokens_str in np.ndenumerate(X)]
        self.d2v_model = Doc2Vec(vector_size=self.vector_size, workers=self.workers, epochs=self.epochs)
        self.d2v_model.build_vocab(tagged_x)
        self.d2v_model.train(tagged_x, total_examples=self.d2v_model.corpus_count, 
                             epochs=self.epochs)

        return self

    def transform(self, X):
        return np.asmatrix(np.array([self.d2v_model.infer_vector(tokens_str.split())
                                     for tokens_str in X]))

In [186]:
%%time

doc2vec_trf = Doc2VecTransformer(epochs=1)
doc2vec_trf.fit(X_train, y_train)

CPU times: user 19.5 s, sys: 2.29 s, total: 21.8 s
Wall time: 17 s


Doc2VecTransformer(epochs=1, vector_size=100)

In [165]:
X_test[0].split()

['7',
 'Coys',
 'of',
 'CRPF',
 'also',
 'comprising',
 'RAF',
 'PERSONNEL',
 'alongwith',
 'relief',
 'materiaal',
 'and',
 '2',
 'doctors',
 'alongwith',
 'medicines,',
 'health-kit',
 'reached',
 'Gujarat.']

In [179]:
inferred_vector = doc2vec_trf.d2v_model.infer_vector(X_test[0].split())
print(inferred_vector.shape)
print(type(inferred_vector))
print(inferred_vector)

inferred_vector2 = np.asmatrix(np.array([doc2vec_trf.d2v_model.infer_vector(X_test[0].split())]))
print(inferred_vector2.shape)
print(type(inferred_vector2))
print(inferred_vector2)

(100,)
<class 'numpy.ndarray'>
[-0.00413006 -0.00258142 -0.00033187  0.00184108 -0.00232771 -0.00613187
  0.00312521  0.00234861  0.00499836  0.00034734 -0.00213006  0.00242959
  0.00362676 -0.0010884  -0.00223558 -0.00527168  0.01529117  0.01042232
  0.00635707 -0.00646272  0.00767631  0.00080644 -0.01011554  0.00401347
  0.00467419 -0.00739857  0.00381793 -0.01005058  0.00343707  0.00968797
  0.00853751  0.00537601 -0.01052791 -0.0122234  -0.00222101 -0.00240534
  0.0068096   0.00554683 -0.00325108 -0.00754306  0.00701016  0.0094353
  0.01977549  0.00567488  0.00351448 -0.00298142 -0.00198921  0.0015138
  0.01099605  0.00284659  0.00921968 -0.00044651 -0.01194185  0.00651459
 -0.01424805 -0.00977723  0.00830761  0.01128369  0.00062722 -0.00690163
 -0.0052788  -0.00534799  0.00604505 -0.0158208  -0.00421478 -0.00799924
 -0.00220149 -0.01180713  0.0114402   0.00591354  0.005495    0.01187458
  0.01173861  0.00303284 -0.00998374 -0.02284285  0.00930111 -0.00858553
  0.00286922 -0.004443

In [172]:
X_train

array(['crudess you are the most charm and charismatic DT member .. thanks for all your videos in Santiago.. and to talk with me twice',
       'Tidal waves triggered by an earthquake off Indonesia on Sunday swept over a vast swath of coastlines, including those in India, Indonesia, Malaysia, the Maldives, Somalia, Sri Lanka and Thailand.',
       'The earthquake happened at 4h34 in the evening and for the night. ',
       ...,
       'Earthquake of the day #Haiti or Google possibly leaving China ?',
       'When the matter had been debated previously, the Non-Aligned Movement foreign ministers firmly stated that there was no right to humanitarian intervention.',
       '@DumboNYC : Oh no RT @endtwist : ConEd power station in #DUMBO is flooding ! #nopower #sandy'],
      dtype=object)

In [173]:
X_test

array(['7 Coys of CRPF also comprising RAF PERSONNEL alongwith relief materiaal and 2 doctors alongwith medicines, health-kit reached Gujarat.',
       'can of olives , can of chickpeas , bag of pasta , bag of hard pretzels , ( also have sneakers I can donate )',
       'We are at the top of Fontamara 43 (area of Carrefour), at the top of the hill named label.. unrecognized characters. We did not get any assistance: water, food, tents, medicine.',
       ...,
       "The country's macroeconomic and fiscal stability has contributed to steady improvement in several economic and social indicators.",
       '* At the regular food aid coordination meeting in Dushanbe, chaired by WFP and attended by a broad representation of NGOs, international agencies and donors, the topic under discussion was how to achieve a better level of mutual understanding with local authorities, particularly in the fields of vulnerable group feeding and land lease projects, where vested interests can be an obstacle

In [174]:
doc2vec_features = doc2vec_trf.transform(X_test)
print(doc2vec_features.shape)
doc2vec_features

(6536, 100)


matrix([[-0.00380614, -0.00182957,  0.00095919, ...,  0.00157907,
          0.00555376, -0.00660127],
        [ 0.00368679,  0.00165392, -0.00248586, ..., -0.00253256,
         -0.00456916,  0.01025566],
        [-0.00382412,  0.00386767, -0.00240853, ...,  0.00302777,
          0.00708908, -0.01281894],
        ...,
        [-0.00404103, -0.00500577,  0.00140955, ..., -0.00319785,
          0.00361738, -0.01487078],
        [-0.00068515, -0.00266437, -0.00105556, ...,  0.00371702,
          0.01842543, -0.03034665],
        [ 0.00245072, -0.00104851, -0.00498233, ..., -0.00018572,
          0.00706883, -0.01178699]], dtype=float32)

In [175]:
%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('w2v_tfidf', Doc2VecTransformer(epochs=1)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

              precision    recall  f1-score   support

   related-0       0.26      0.86      0.40      1532
   related-1       0.86      0.12      0.21      4950

   micro avg       0.33      0.29      0.31      6482
   macro avg       0.56      0.49      0.30      6482
weighted avg       0.71      0.29      0.26      6482

              precision    recall  f1-score   support

   request-0       0.83      0.94      0.88      5393
   request-1       0.24      0.09      0.13      1143

    accuracy                           0.79      6536
   macro avg       0.53      0.51      0.50      6536
weighted avg       0.73      0.79      0.75      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.26      0.42      6506
     offer-1       0.00      0.77      0.01        30

    accuracy                           0.27      6536
   macro avg       0.50      0.51      0.21      6536
weighted avg       0.99      0.27      0.41      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

 transport-0       0.95      0.98      0.97      6219
 transport-1       0.11      0.04      0.06       317

    accuracy                           0.94      6536
   macro avg       0.53      0.51      0.51      6536
weighted avg       0.91      0.94      0.92      6536

              precision    recall  f1-score   support

 buildings-0       0.95      0.98      0.97      6209
 buildings-1       0.17      0.06      0.09       327

    accuracy                           0.94      6536
   macro avg       0.56      0.52      0.53      6536
weighted avg       0.91      0.94      0.92      6536

               precision    recall  f1-score   support

electricity-0       0.98      0.99      0.99      6414
electricity-1       0.08      0.05      0.06       122

     accuracy                           0.97      6536
    macro avg       0.53      0.52      0.52      6536
 weighted avg       0.97      0.97      0.97      6536

             

### 8. Extra Feature - Distance Between Message Word Vector to Categories Word Vectors 

We will use Glove as our space vectorizer as it is trained on a broad corpus and thus able to better describe texts / words

In [191]:
category_columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [207]:
categories_tokens = np.array([np.array(cat.split('_')) for cat in category_columns])

In [208]:
categories_tokens

array([array(['related'], dtype='<U7'), array(['request'], dtype='<U7'),
       array(['offer'], dtype='<U5'),
       array(['aid', 'related'], dtype='<U7'),
       array(['medical', 'help'], dtype='<U7'),
       array(['medical', 'products'], dtype='<U8'),
       array(['search', 'and', 'rescue'], dtype='<U6'),
       array(['security'], dtype='<U8'), array(['military'], dtype='<U8'),
       array(['child', 'alone'], dtype='<U5'),
       array(['water'], dtype='<U5'), array(['food'], dtype='<U4'),
       array(['shelter'], dtype='<U7'), array(['clothing'], dtype='<U8'),
       array(['money'], dtype='<U5'),
       array(['missing', 'people'], dtype='<U7'),
       array(['refugees'], dtype='<U8'), array(['death'], dtype='<U5'),
       array(['other', 'aid'], dtype='<U5'),
       array(['infrastructure', 'related'], dtype='<U14'),
       array(['transport'], dtype='<U9'),
       array(['buildings'], dtype='<U9'),
       array(['electricity'], dtype='<U11'),
       array(['tools'], dtype

In [209]:
glove_w2v_mean = MeanEmbeddingTrainVectorizer(glove_w2v)

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [210]:
glove_w2v_mean.fit(categories_tokens, None)

In [213]:
cat_tokens_embeddings = glove_w2v_mean.transform(categories_tokens)

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [215]:
print(cat_tokens_embeddings.shape)
print(cat_tokens_embeddings)

(36, 50)
[[ 8.70540023e-01  2.68770009e-01 -2.48119995e-01 ...  9.47820008e-01
   8.64360034e-02  3.95020004e-03]
 [ 8.19299996e-01 -1.92120001e-01 -1.41980007e-01 ...  1.06869996e+00
  -1.96319997e-01  4.44750011e-01]
 [ 1.06850004e+00  6.77020013e-01  1.27279997e-01 ...  1.53380001e+00
  -3.31820011e-01  4.05580014e-01]
 ...
 [ 6.57759979e-02  5.00930011e-01 -5.60519993e-01 ... -5.54250002e-01
  -6.59200013e-01  6.80409968e-02]
 [ 3.00691485e-01  1.75524995e-01  1.01567507e-02 ...  1.53229997e-01
   1.50001049e-03  2.48915002e-01]
 [ 6.12894952e-01  5.31800017e-02  3.29084992e-01 ...  1.27181995e+00
   3.28752995e-01  2.28199989e-01]]


In [221]:
cat_tokens_embeddings[0]

array([ 0.87054002,  0.26877001, -0.24811999,  0.77732998, -0.10663   ,
        0.98853999,  0.21523   , -0.9465    ,  0.45146999,  0.076455  ,
       -0.089917  ,  0.34068   , -0.021292  , -0.40380001,  0.85165   ,
       -0.22600999, -0.69112998, -0.051165  ,  0.18378   , -0.15451001,
        0.54206002, -0.44464001,  0.44811001, -0.26600999, -0.38775   ,
       -1.10669994, -0.43698001, -0.68585998,  0.14373   ,  0.24357   ,
        3.34319997, -0.42392001,  0.42776   , -1.08829999,  0.19154   ,
        0.20925   , -0.2615    , -0.43105   , -0.036849  ,  0.66139001,
       -0.73272997, -0.13605   ,  0.88170999,  0.84206003,  0.67846   ,
       -0.010663  , -0.1246    ,  0.94782001,  0.086436  ,  0.0039502 ])

In [218]:
test_msgs_embeddings = glove_w2v_mean.transform(X_test)

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [219]:
print(test_msgs_embeddings.shape)
print(test_msgs_embeddings)

(6536, 50)
[[-0.09849867  0.80379105  0.58723372 ... -0.41366184  0.31295398
   0.95703429]
 [-0.07353351  0.79315662  0.5192886  ... -0.3660782   0.43199539
   0.83800238]
 [-0.06318152  0.77121454  0.58120042 ... -0.41525692  0.30183759
   0.95767117]
 ...
 [-0.08637894  0.79725534  0.5465371  ... -0.47110757  0.33932394
   0.90649241]
 [-0.06331346  0.78136551  0.62817228 ... -0.40795022  0.33388904
   1.01786113]
 [-0.09921126  0.79420722  0.63790202 ... -0.43955088  0.36906043
   0.97980541]]


In [220]:
test_msgs_embeddings[0]

array([-0.09849867,  0.80379105,  0.58723372,  0.63757902,  0.39040658,
        0.06465339,  0.30373609, -0.7160418 , -0.27188271, -0.02515468,
       -0.16128114,  0.31148669, -0.48598075, -0.42087334, -0.0084684 ,
       -0.10716169, -0.20211704, -0.00920699, -0.42126659, -0.34465602,
       -0.36295775, -0.25602722,  0.63614345,  0.36809188, -0.17958124,
       -0.81298083, -0.30120522, -0.02328389,  0.15080453, -0.24925278,
        2.60491633, -0.04956524, -0.29668048,  0.54801393,  0.14861742,
       -0.74064833,  0.53189814, -0.36782199,  0.34614521,  0.07964083,
        0.70820642,  0.09071849, -0.22079436, -0.04370295, -0.05506461,
        0.30032167,  0.1601108 , -0.41366184,  0.31295398,  0.95703429])

In [285]:
from sklearn.metrics.pairwise import cosine_similarity

def compute_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1, feature_vec_2.reshape(1, -1))[0][0]

In [293]:
cat_tokens_embeddings

array([[ 8.70540023e-01,  2.68770009e-01, -2.48119995e-01, ...,
         9.47820008e-01,  8.64360034e-02,  3.95020004e-03],
       [ 8.19299996e-01, -1.92120001e-01, -1.41980007e-01, ...,
         1.06869996e+00, -1.96319997e-01,  4.44750011e-01],
       [ 1.06850004e+00,  6.77020013e-01,  1.27279997e-01, ...,
         1.53380001e+00, -3.31820011e-01,  4.05580014e-01],
       ...,
       [ 6.57759979e-02,  5.00930011e-01, -5.60519993e-01, ...,
        -5.54250002e-01, -6.59200013e-01,  6.80409968e-02],
       [ 3.00691485e-01,  1.75524995e-01,  1.01567507e-02, ...,
         1.53229997e-01,  1.50001049e-03,  2.48915002e-01],
       [ 6.12894952e-01,  5.31800017e-02,  3.29084992e-01, ...,
         1.27181995e+00,  3.28752995e-01,  2.28199989e-01]])

In [313]:
print(cat_tokens_embeddings.shape)
print(test_msgs_embeddings[:10].shape)

(36, 50)
(10, 50)


In [301]:
M = np.matrix([[1], [2], [3], [4]])
M

matrix([[1],
        [2],
        [3],
        [4]])

In [303]:
A = np.array(M.T)[0]
A

array([1, 2, 3, 4])

In [304]:
np.asarray(M).reshape(-1)

array([1, 2, 3, 4])

In [299]:
vec1 = np.array([[1,1,0,1,1]])
vec2 = np.array([[0,1,0,1,1]])
#print(cosine_similarity([vec1, vec2]))
print(cosine_similarity(vec1, vec2))

[[0.8660254]]


AttributeError: 'NoneType' object has no attribute 'A1'

In [322]:
print(test_msgs_embeddings[:10].shape)
print(cat_tokens_embeddings.shape)
print(cosine_similarity(test_msgs_embeddings[:10],cat_tokens_embeddings).shape)

(10, 50)
(36, 50)
(10, 36)


In [317]:
from sklearn.metrics.pairwise import cosine_similarity

class CategoriesSimilarity(object):
    
    def __init__(self, word2vec_model, categories_vectors):
        self.word2vec_model = word2vec_model
        self.categories_vectors = categories_vectors
        self.num_dims = word2vec_model.vector_size
        
    def cos_sim_(feature_vec_1, feature_vec_2):    
        return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]
            
    def fit(self, X, y):            
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            cats_similarities = np.empty([self.categories_vectors.shape[0]])
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        cats_similarities = cosine_similarity(mean_embeddings, self.categories_vectors)
            
        return cats_similarities

In [318]:
cats_sim_transf = CategoriesSimilarity(glove_w2v, cat_tokens_embeddings)

In [319]:
cats_sim_transf.fit(X_test, None)

In [325]:
cats_sim_transf.transform(X_test)

array([[0.49156446, 0.41961395, 0.51883279, ..., 0.37765505, 0.52376198,
        0.54196545],
       [0.50996024, 0.42967021, 0.51640248, ..., 0.3675782 , 0.52620765,
        0.56729783],
       [0.50943892, 0.44520025, 0.5291723 , ..., 0.37118992, 0.53494012,
        0.56678289],
       ...,
       [0.474987  , 0.42910571, 0.51644435, ..., 0.36782292, 0.51544412,
        0.53418885],
       [0.47501837, 0.42284328, 0.50677142, ..., 0.3436368 , 0.49888376,
        0.53266288],
       [0.46621487, 0.40606677, 0.49145115, ..., 0.34584991, 0.49441343,
        0.52072637]])

In [326]:
%%time

from sklearn.pipeline import FeatureUnion

all_feats = Pipeline([
    ('cats_sim', CategoriesSimilarity(glove_w2v, cat_tokens_embeddings)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

all_feats.fit(X_train, y_train)

y_pred = all_feats.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

              precision    recall  f1-score   support

   related-0       0.41      0.07      0.12      1532
   related-1       0.77      0.94      0.85      4950

   micro avg       0.75      0.74      0.74      6482
   macro avg       0.59      0.51      0.48      6482
weighted avg       0.68      0.74      0.67      6482

              precision    recall  f1-score   support

   request-0       0.83      0.95      0.89      5393
   request-1       0.28      0.09      0.14      1143

    accuracy                           0.80      6536
   macro avg       0.56      0.52      0.51      6536
weighted avg       0.74      0.80      0.76      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.25      0.39      6506
     offer-1       0.01      0.90      0.01        30

    accuracy                           0.25      6536
   macro avg       0.50      0.57      0.20      6536
weighted avg       0.99      0.25      0.39      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

 transport-0       0.97      0.17      0.29      6219
 transport-1       0.05      0.91      0.10       317

    accuracy                           0.21      6536
   macro avg       0.51      0.54      0.20      6536
weighted avg       0.93      0.21      0.28      6536

              precision    recall  f1-score   support

 buildings-0       0.97      0.18      0.30      6209
 buildings-1       0.05      0.90      0.10       327

    accuracy                           0.21      6536
   macro avg       0.51      0.54      0.20      6536
weighted avg       0.92      0.21      0.29      6536

               precision    recall  f1-score   support

electricity-0       0.99      0.21      0.35      6414
electricity-1       0.02      0.84      0.04       122

     accuracy                           0.22      6536
    macro avg       0.50      0.53      0.19      6536
 weighted avg       0.97      0.22      0.34      6536

             

### 8. NLP Feature Selection using Sklearn Pipelines

#### 8.1 - All features together

In [329]:
%%time

from sklearn.pipeline import FeatureUnion

all_feats = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_w2v)),
        ('doc2vec', Doc2VecTransformer(epochs=1)),
        ('cats_sim', CategoriesSimilarity(glove_w2v, cat_tokens_embeddings))
    ])),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

all_feats.fit(X_train, y_train)

y_pred = all_feats.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


              precision    recall  f1-score   support

   related-0       0.29      0.53      0.38      1532
   related-1       0.81      0.57      0.67      4950

   micro avg       0.58      0.56      0.57      6482
   macro avg       0.55      0.55      0.52      6482
weighted avg       0.69      0.56      0.60      6482

              precision    recall  f1-score   support

   request-0       0.83      0.96      0.89      5393
   request-1       0.28      0.08      0.12      1143

    accuracy                           0.80      6536
   macro avg       0.55      0.52      0.51      6536
weighted avg       0.73      0.80      0.76      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.38      0.55      6506
     offer-1       0.01      0.80      0.01        30

    accuracy                           0.38      6536
   macro avg       0.50      0.59      0.28      6536
weighted avg       0.99      0.38      0.54      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

 transport-0       0.96      0.87      0.91      6219
 transport-1       0.08      0.22      0.12       317

    accuracy                           0.84      6536
   macro avg       0.52      0.55      0.52      6536
weighted avg       0.91      0.84      0.87      6536

              precision    recall  f1-score   support

 buildings-0       0.95      0.91      0.93      6209
 buildings-1       0.09      0.18      0.12       327

    accuracy                           0.87      6536
   macro avg       0.52      0.54      0.53      6536
weighted avg       0.91      0.87      0.89      6536

               precision    recall  f1-score   support

electricity-0       0.98      0.95      0.97      6414
electricity-1       0.04      0.11      0.06       122

     accuracy                           0.94      6536
    macro avg       0.51      0.53      0.51      6536
 weighted avg       0.96      0.94      0.95      6536

             

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize, ngram_range=(1,3))),
    ('tfidf', TfidfTransformer()),
    ('multi_clf', MultiOutputClassifier(RandomForestClassifier(random_state=199), n_jobs=-1))
])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
X_train.shape

(19662,)

In [14]:
X_train

array(['It works as a fertilizer enhancement, a composting additive, and even as an alternative to household cleaning products, according to EMRO.',
       'I need a job to feed my family. I am an electrician and a musician. I perform harmonization and orchestration in Mexico. I speak English and French. Let me know how I can help',
       'The report also warns that if the clan fighting extends to sorghum-producing areas in the south during this critical stage of crop harvest, it could further hamper harvesting and exert a negative impact on food security throughout Somalia.',
       ...,
       'HELP THE EARTHQUAKE VICTIMS IN HAITI http tinyurl.com yk3bspe links to many resources where you can make donations. Broke? donate time',
       'The low snowfall pattern and the snowfall differentiation among catchments are clearly reflected in the irrigated wheat production levels.',
       "Having fun at #Lexi 's ! Real food in real house with electrical lighting . Good times . I feel human

In [15]:
# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

ValueError: WRITEBACKIFCOPY base is read-only

In [ ]:
print(X_test.shape)

In [ ]:
print(y_pred.shape)
y_pred

In [ ]:
y_pred[:,0]

In [ ]:
y_test

In [ ]:
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'multi_clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'multi_clf__estimator__bootstrap', 'multi_clf__estimator__class_weight', 'multi_clf__estimator__criterion', 'multi_clf__estimator__max_depth', 'multi_clf__estimator__max_features', 'multi_clf__estimator__max_leaf_nodes', 'multi_clf__estimator__min_impurity_decrease', 'multi_clf__estimator__min_impurity_split', 'multi_clf__estimator__min_samples_leaf', 'multi_clf__estimator__min_samples_split', 'multi_clf__estimator__min_weight_fraction_leaf', 'multi_clf__estimator__n_estimators', 'multi_clf__estimator__n_jobs', 'multi_clf__estimator

In [ ]:
parameters = {
        'multi_clf__estimator__n_estimators': [20,50],
        'multi_clf__estimator__max_depth': [50, 100]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [31]:
# train classifier
cv.fit(X_train, y_train)

# predict on test data
y_pred = cv.predict(X_test)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV]  ................................................................
[CV] ...................... , score=0.22017088800732376, total= 2.7min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min remaining:    0.0s


[CV] ...................... , score=0.22978333841928594, total= 2.4min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.7min remaining:    0.0s


[CV] ...................... , score=0.22642660970399756, total= 2.5min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.4min finished


In [32]:
cv.best_params_

{}

In [33]:
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

             precision    recall  f1-score   support

  related-0       0.55      0.32      0.41      1565
  related-1       0.81      0.91      0.86      4941

avg / total       0.74      0.77      0.75      6506

             precision    recall  f1-score   support

  request-0       0.89      0.98      0.93      5420
  request-1       0.81      0.39      0.53      1134

avg / total       0.87      0.88      0.86      6554

             precision    recall  f1-score   support

    offer-0       1.00      1.00      1.00      6524
    offer-1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6554

               precision    recall  f1-score   support

aid_related-0       0.71      0.89      0.79      3900
aid_related-1       0.75      0.46      0.57      2654

  avg / total       0.73      0.72      0.70      6554

                precision    recall  f1-score   support

medical_help-0       0.93      1.00      0.96      6038
medical_help-1     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [122]:
# Output a pickle file for the model
joblib.dump(cv, 'classifier.pkl') 

['initial_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.